# Torch bridges examples

## Homogeneous dataset

In [ ]:
import copy
import pickle

import numpy as np
from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid.containers.sample import Sample
from plaid_ops.mesh.feature_engineering import update_dataset_with_sdf
from plaid_ops.mesh.transformations import (
    compute_bounding_box,
    project_on_regular_grid,
)
from torch.utils.data import DataLoader

from plaid_bridges.common import HomogeneousDataset
from plaid_bridges.torch import GridFieldsAndScalarsDataset

In [ ]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]

dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

In [ ]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [
    f for f in all_feat_ids if "field" in f.values() and "Base_2_2" in f.values()
]

in_features_identifiers = scalar_features
out_features_identifiers = field_features

print(in_features_identifiers)
print(out_features_identifiers)

In [ ]:
homogen_dataset = HomogeneousDataset(
    dataset=dataset,
    in_features_identifiers=in_features_identifiers,
    out_features_identifiers=out_features_identifiers,
)

homogen_dataset.show_details()

In [ ]:
loader = DataLoader(
    homogen_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = out_features_identifiers[0]
before = copy.deepcopy(dataset[ids[1]].get_feature_from_identifier(out_feat_id))

predictions = []
for batch in loader:
    for torch_sample in batch:
        predictions.append(torch_sample[1])

dataset_pred = homogen_dataset.inverse_transform(predictions)

after = copy.deepcopy(dataset_pred[ids[1]].get_feature_from_identifier(out_feat_id))

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

## Projection on constant rectilinear grid, with scalars as constant fields

In [ ]:
hf_dataset = load_dataset(
    "PLAID-datasets/2D_Multiscale_Hyperelasticity", split="all_samples"
)
pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids_train = pb_def.get_split("DOE_train")[:10]

dataset_train, _ = huggingface_dataset_to_plaid(
    hf_dataset, ids=ids_train, processes_number=5
)
n_train = len(dataset_train)
print(dataset_train)

In [ ]:
dims = (101, 101)
dataset_train = update_dataset_with_sdf(dataset_train, verbose=True)

print(dataset_train.get_sample_ids())

bbox = compute_bounding_box(dataset_train)
projected_dataset_train = project_on_regular_grid(
    dataset_train, dimensions=dims, bbox=bbox, verbose=True
)

all_feat_ids = dataset_train[ids_train[0]].get_all_features_identifiers()
scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

In [ ]:
torch_dataset = GridFieldsAndScalarsDataset(
    dataset=projected_dataset_train,
    dimensions=dims,
    in_features_identifiers=[field_features[0], scalar_features[0]],
    out_features_identifiers=[scalar_features[1], field_features[1]],
)

loader = DataLoader(
    torch_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = scalar_features[0]

before = copy.deepcopy(
    projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

predictions = []
for batch in loader:
    for torch_sample in batch[1]:
        predictions.append(torch_sample.detach().cpu().numpy())

projected_dataset_train = torch_dataset.inverse_transform(predictions)

after = copy.deepcopy(
    projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))